# 🎯 CMake Nedir?

* CMake, C++ projeleri için bir derleme yönetim ve yapılandırma sistemidir.

İşlevi şu:

**Kaynak kodunu,**
***üstüne hangi kütüphanelerin ekleneceğini,**
**hangi derleyici seçeneklerinin kullanılacağını,**
**nasıl linkleneceğini**
tanımlarsın →
**CMake bunu alır, platformuna uygun build dosyalarına dönüştürür.**

* Yani C++ projelerinin “derleme sürecini” yöneten bir orkestratör.

## 🧠 Peki neden gerekiyor? Basit C++ dosyalarını da g++ ile derleriz?

* Doğru:

>g++ main.cpp -o program
>küçük scriptler için yeterli.

Ama şimdi düşün:

- Birden fazla .cpp dosyası var

* 10 tane include klasörü var

* OpenCV’yi linklemen gerekiyor

* LibTorch’u linklemen gerekiyor

* CUDA varsa ekstra library path’leri var

* Windows, Linux, macOS’ta derlemek istiyorsun

* Debug/Release build farkları var

* Farklı derleyici seçenekleri var

#### Bu noktada “g++ komutu” şöyle bir şeye dönüşüyor:

```bash
g++ main.cpp utils.cpp math.cpp \
    -I include \
    -I /usr/local/include/opencv4 \
    -I /usr/local/libtorch/include \
    -L /usr/local/lib -lopencv_core -lopencv_imgproc \
    -L /usr/local/libtorch/lib -ltorch_cpu -lc10 \
    -O3 -Wall -Wextra -pthread \
    -o my_app
```

* Bu komutu kim yazmak ister?
* Kim takip eder?
* Hangi sistemde ne değişti?

İşte CMake burada oyuna giriyor.

# ⚙️ CMake ne yapar?

* CMake bir yapı tanımı alır:
```cmake
add_executable(my_app
    src/main.cpp
    src/utils.cpp
)

target_link_libraries(my_app PRIVATE
    ${OpenCV_LIBS}
    ${TORCH_LIBRARIES}
)
```



* Sen sadece bu tanımı yazarsın.

#### CMake bunu platforma göre dönüştürür:

* Linux → Makefile / Ninja

* Windows → Visual Studio solution

* macOS → Xcode project

#### Sonra da:
```bash
cmake ..
cmake --build .
```


dediğinde CMake:

* derle,

* linkle,

* include path’lerini ayarla,

* kütüphaneleri bul ve bağla

işlemlerini tam otomatik yapar.

# 🧱 CMake’in En Büyük Avantajları
### 1) Platform bağımsız

* Aynı CMakeLists.txt → hem Windows hem Linux hem macOS’ta çalışır.

### 2) Kütüphane bulmayı otomatik yapar

OpenCV, LibTorch, CUDA gibi büyük kütüphaneleri:

* konumunu bulur

* include path’lerini ekler

lib dosyalarını linkler

* gerekli derleme seçeneklerini ayarlar

>Bu yüzden profesyonel projelerde “g++ tek başına” görülmez.

### 🔍 Gerçek örnek: LibTorch neden CMake istiyor?

PyTorch’un C++ versiyonu (LibTorch) şu satırı ister:
```bash
find_package(Torch REQUIRED)
target_link_libraries(my_app PRIVATE ${TORCH_LIBRARIES})
```


Çünkü LibTorch:

* bir sürü .so / .dll içerir

* include klasörleri çok karmaşıktır

- CUDA varsa daha da karmaşık hale gelir

* CMake bunları tek bir komutla çözer.

## 🔍 OpenCV neden CMake istiyor?

Aynı mantık:
```bash
find_package(OpenCV REQUIRED)
target_link_libraries(my_app PRIVATE ${OpenCV_LIBS})
```


Bu satır:

* OpenCV’nin nerede olduğunu otomatik bulur

* doğru lib dosyalarıyla linkler

* include path’lerini ayarlar

Manuel yapmaya kalkarsan kabus olur.

----

# 🔧 CMake’in Temel Taşları

CMakeLists.txt içinde en çok göreceğin komutlar şunlar:

* cmake_minimum_required

* project

* set (özellikle C++ standardı için)

* add_executable

* add_library

* target_include_directories

* target_link_libraries

* find_package

* message (debug print)

Bu sekizli, C++ projelerinde her gün göreceğin çekirdek komutlar.

## 1️⃣ cmake_minimum_required(VERSION …)

Bu:

* “Bu projeyi minimum şu CMake versiyonuyla çalıştır.”

Yazımı:
```bash
cmake_minimum_required(VERSION 3.16)
```


Neden 3.16?

* LibTorch → minimum 3.1 istiyor ama 3.16 güvenli bir sınır

- OpenCV modern sürümleri için de gayet uygun

* Linux/Windows ortamlarında stabil çalışıyor

Bu satır her CMakeLists.txt'nin en üstünde bulunmalı.

## 2️⃣ project(ProjectName LANGUAGES CXX)

Projenin adı + kullanılan diller:
```bash
project(MyApp LANGUAGES CXX)
```


Bu ne işe yarıyor?

* Visual Studio veya CLion açtığında projenin adını gösteriyor

* CMake'e “Bu projede C++ kullanılacak” demiş oluyorsun

* C++ seçmezsen bazı özellikler devreye girmez

## 3️⃣ C++ standardı ayarlamak → set(CMAKE_CXX_STANDARD 17)

**Bu en kritik komutlardan biri.**
```cpp
set(CMAKE_CXX_STANDARD 17)
set(CMAKE_CXX_STANDARD_REQUIRED ON)
set(CMAKE_CXX_EXTENSIONS OFF)
```


Bunlar ne demek?

* 17 → C++17 kullan

* REQUIRED ON → başka standarda izin verme

* EXTENSIONS OFF → gcc/clang’a özgü uzantıları kapat (taşınabilirlik artar)

* LibTorch, C++17 ile çok stabil.

Birçok derste de hep C++17 ya da C++20 kullanacağız.

## 4️⃣ add_executable → Program oluşturmak

* C++’ta “program” = “executable”.

CMake’te şu şekilde oluşturursun:
```cpp
add_executable(my_program
    src/main.cpp
    src/utils.cpp
)
```


Bu satır:

* my_program adında bir target oluşturur

* Bu target, listelenen .cpp dosyalarından derlenir

* Bu target’ın ismi sayesinde sonra:
```cpp
target_link_libraries(my_program ...)
target_include_directories(my_program ...)
```


* gibi işlemler yaparsın.

## 5️⃣ add_library → Kütüphane oluşturmak

* Bazı projelerde kendi .so (Linux) / .dll (Windows) / .a (static) kütüphaneni üretirsin.

Örneğin:
```cpp
add_library(my_lib STATIC
    src/math.cpp
    src/helpers.cpp
)
```


veya:
```cpp
add_library(my_lib SHARED
    src/math.cpp
)
```


Bu kütüphaneyi:
```cpp
target_link_libraries(my_program PRIVATE my_lib)
```


ile programa bağlarsın.

**Biz LibTorch/OpenCV tarafında genelde hazır kütüphaneye bağlanacağız, ama kendi kütüphaneni yapmak da çok sık görülen bir durum.**

## 6️⃣ target_include_directories → Header dosyalarını tanıtmak

C++’ta:
```cpp
#include "math.h"
```


* dediğin dosya neredeyse, CMake onu include path’e eklemeli.

Örnek:
```cpp
target_include_directories(my_program
    PRIVATE
        ${CMAKE_SOURCE_DIR}/include
)

```

Bu, derleyiciye:
```bash
-I <proje_kök>/include
```


* parametresini ekler.

LibTorch/OpenCV için genelde:
```cpp
target_include_directories(my_program PRIVATE ${OpenCV_INCLUDE_DIRS})
```

kullanacağız.

## 7️⃣ target_link_libraries → Kütüphane bağlamak

* En önemli komutlardan biri.
- LibTorch, OpenCV, pthread, CUDA hepsi buraya bağlanacak.

Örnek:
```cpp
target_link_libraries(my_program
    PRIVATE
        ${OpenCV_LIBS}
        ${TORCH_LIBRARIES}
)
```


Bu satır:

* OpenCV kütüphanelerini (opencv_core, opencv_imgproc …)

* LibTorch kütüphanelerini (torch_cpu, c10, torch_cuda)
bağlar.

Bu olmazsa “undefined reference” hatalarını görürsün.

## 8️⃣ find_package

**Bu komut CMake’in sihridir.**

CMake’te:
```cpp
find_package(OpenCV REQUIRED)
find_package(Torch REQUIRED)
```


yazınca:

* OpenCV / LibTorch bulundu

* include path’leri hazır

* library path’leri hazır

* linklenecek lib listeleri hazır

* derleyici ayarları eklendi

>C++ büyük kütüphaneler artık “bir satırla” projeye girmiş olur.

Yani:
>OpenCV’yi elle bulamazsın. Torch’u elle bulamazsın. find_package olmadan iş yok.

## 9️⃣ message() → Debug print

CMake içinde debug amaçlı:
```cpp
message("OpenCV bulundu!")
```


veya önemli değerleri yazdırmak için:
```cpp
message("TORCH_LIBRARIES = ${TORCH_LIBRARIES}")
message("OpenCV include dizini = ${OpenCV_INCLUDE_DIRS}")
```


**Konsolda CMake configure sırasında görürsün.**

## 🔥 Özet

CMake’in çekirdek mantığı:

**project() → Projeyi tanıt**

**add_executable() → Program hedefi oluştur**

**add_library() → Kütüphane oluştur (gerekirse)**

**target_include_directories() → Header dosyalarını tanıt**

**target_link_libraries() → Kütüphaneleri bağla**

**find_package() → OpenCV / LibTorch / CUDA gibi kütüphaneleri bul**

**set(... CXX_STANDARD ... ) → C++ standardını belirle**